In [1]:
import numpy as np
import cv2
import glob
from icecream import ic
from pickle import dump, load
from copy import deepcopy
from scipy.spatial.transform import Rotation
from IPython.display import clear_output

Checkerboard and criteria for calibration

In [2]:
CHECKERBOARD = (10,7)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

Get calibration images

In [ ]:
# Open the right camera
cam = cv2.VideoCapture(0)

cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280 )
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cam.get(cv2.CAP_PROP_FPS))

ic(frame_width, frame_height, fps)

n = 0

while True:
    ret, raw = cam.read()

    gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
    frame = deepcopy(raw)
    
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    if ret:
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)

        # Draw and display the corners
        frame = cv2.drawChessboardCorners(frame, CHECKERBOARD, corners2, ret)
       
    cv2.imshow('img',frame)
    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break
    elif cv2.waitKey(1) == ord('c'):
        if ret:
            cv2.imwrite('calibration images/{}.png'.format(n), raw)
            ic(n)
            n += 1

# Release the capture and writer objects
cam.release()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread("calibration images/picamv3/LEFT/Left15.png")
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
frame = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
cv2.imshow('img',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

ic(corners2)

Calibration 2D
Right

In [34]:
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= 2.4
prev_img_shape = None


# Extracting path of individual image stored in a given directory
images = glob.glob('./calibration images/picamv3/RIGHT/*.png')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    cv2.imshow('img',img)
    cv2.waitKey(1)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]

In [35]:
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtxR, distR, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
ic(mtxR, distR)

with open('calibrationR.calib', 'wb') as f:
    obj = (mtxR,distR)
    dump(obj, f)

ic| mtxR: array([[932.53891575,   0.        , 272.58233251],
                 [  0.        , 927.47669794, 337.06310409],
                 [  0.        ,   0.        ,   1.        ]])
    distR: array([[-0.01396712,  0.35858365,  0.00813819,  0.01383208, -1.00899726]])


Calibration 2D
Left

In [36]:
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = []

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= 2.4
prev_img_shape = None

# Extracting path of individual image stored in a given directory
images = glob.glob('./calibration images/picamv3/LEFT/*.png')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

    cv2.imshow('img', img)
    cv2.waitKey(1)

cv2.destroyAllWindows()

h, w = img.shape[:2]

In [37]:
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtxL, distL, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
ic(mtxL, distL)

with open('calibrationL.calib', 'wb') as f:
    obj = (mtxL, distL)
    dump(obj, f)

ic| mtxL: array([[931.30019496,   0.        , 281.76963219],
                 [  0.        , 926.8736945 , 335.41083851],
                 [  0.        ,   0.        ,   1.        ]])
    distL: array([[ 0.07865313, -0.84050108,  0.01016875,  0.01707246,  3.63095832]])


Calibration 3d

In [39]:
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []

imgpointsR = []
imgpointsL = []

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= 2.4
prev_img_shape = None

# Extracting path of individual image stored in a given directory
imagesR = glob.glob('./calibration images/picamv3/STEREO/Right*.png')
imagesL = glob.glob('./calibration images/picamv3/STEREO/Left*.png')

for fnameR, fnameL in zip(imagesR, imagesL):
    imgR = cv2.imread(fnameR)
    imgL = cv2.imread(fnameL)
    
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    retR, cornersR = cv2.findChessboardCorners(grayR, CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    retL, cornersL = cv2.findChessboardCorners(grayL, CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if retR and retL:
        objpoints.append(objp)
        
        # refining pixel coordinates for given 2d points.
        corners2R = cv2.cornerSubPix(grayR, cornersR, (11, 11), (-1, -1), criteria)
        corners2L = cv2.cornerSubPix(grayL, cornersL, (11, 11), (-1, -1), criteria)

        imgpointsR.append(corners2R)
        imgpointsL.append(corners2L)

        # Draw and display the corners
        imgR = cv2.drawChessboardCorners(imgR, CHECKERBOARD, corners2R, retR)
        imgL = cv2.drawChessboardCorners(imgL, CHECKERBOARD, corners2L, retL)
    
    cv2.imshow('img', np.concatenate((imgL, imgR), axis=1))
    cv2.waitKey(1)

cv2.destroyAllWindows()

h, w = imgR.shape[:2]

In [41]:
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, KL, distL, KR, distR, R, T, E, F = cv2.stereoCalibrate(objpoints, imgpointsL, imgpointsR, mtxL, distL,
                                                                 mtxR, distR, (w, h), criteria = criteria, flags = cv2.CALIB_FIX_INTRINSIC)
ic(KL, distL, KR, distR)
ic(R, T)

with open('calibrationStereo.calib', 'wb') as f:
    obj = (mtxR, distR, mtxL, distL, R, T)
    dump(obj, f)

ic| KL: array([[931.30019496,   0.        , 281.76963219],
               [  0.        , 926.8736945 , 335.41083851],
               [  0.        ,   0.        ,   1.        ]])
    distL: array([[ 0.07865313, -0.84050108,  0.01016875,  0.01707246,  3.63095832]])
    KR: array([[932.53891575,   0.        , 272.58233251],
               [  0.        , 927.47669794, 337.06310409],
               [  0.        ,   0.        ,   1.        ]])
    distR: array([[-0.01396712,  0.35858365,  0.00813819,  0.01383208, -1.00899726]])
ic| R: array([[ 0.9987424 , -0.0226302 , -0.04473809],
              [ 0.02377945,  0.9993964 ,  0.02532525],
              [ 0.04413797, -0.02635725,  0.99867769]])
    T: array([[-11.85573165],
              [ -0.37841502],
              [ -0.13359546]])


Egomotion Calculations

In [ ]:
# Load the Calibration
with open('calibration_logi.calib', 'rb') as f:
    mtx,dist = load(f)
    ic(mtx, dist)

In [ ]:
#Optical FLow Parameters
lk_of_params = dict( winSize  = (5, 5),
                  maxLevel = 10,
                  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initiate FAST object
fast = cv2.FastFeatureDetector_create(threshold=7, nonmaxSuppression=True)

start cam

In [ ]:
# Open the default camera
cam = cv2.VideoCapture(0)

cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280 )
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cam.get(cv2.CAP_PROP_FPS))

ic(frame_width, frame_height, fps)

In [ ]:
Rpose = np.eye(3)
Tpose = np.zeros(shape=(3,1))

ret, raw = cam.read()

f1 = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)

while True:
    f0 = deepcopy(f1) 
    ret, raw = cam.read()
    frame = raw
    f1 = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
    
    if good_p1.size / 2 < 5000:
        p0 = fast.detect(f0)
        p0 = np.array([x.pt for x in p0], dtype=np.float32).reshape(-1, 1, 2)
    else:
        p0 = deepcopy(good_p1)
        p0 = np.array(p0, dtype=np.float32).reshape(-1, 1, 2)
    
    # Calculate optical flow between frames, st holds status
    # of points from frame to frame
    p1, st, err = cv2.calcOpticalFlowPyrLK(f0, f1, p0, None, **lk_of_params)
    
    if p1 is not None:
        good_p0 = p0[st == 1]
        good_p1 = p1[st == 1]
    else:
        good_p0 = np.empty((0, 2), dtype=np.float32)

    if good_p0.size != 0:
        #Find essential matrix and decompose
        E, _ = cv2.findEssentialMat(good_p1, good_p0, mtx, cv2.RANSAC, 0.999, 1.0, None)
        if E is not None:
            if not np.isnan(E).any() and E.size == 9:
                _, R, t, _ = cv2.recoverPose(E, good_p0, good_p1, mtx)
                
                Rmag = abs(Rotation.from_matrix(R).as_euler('xyz', degrees=True))
                if Rmag.max() < 100:       # Rotation Threshold
                    if abs(t).max() > 0.5:
                        Tpose = Tpose + np.linalg.inv(Rpose) @ t
                    Rpose = R @ Rpose
        
        for i in range(len(good_p0)):
            cv2.circle(frame,(int(good_p0[i][0]),int(good_p0[i][1])),1,(255,0,0))
            cv2.circle(frame,(int(good_p1[i][0]),int(good_p1[i][1])),1,(0,0,255))
        
        cv2.putText(frame, "PITCH{0:.4f} YAW{1:.4f} ROLL{2:.4f}".format(*Rotation.from_matrix(Rpose).as_euler("xyz",degrees=True)), (100,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0))
        cv2.putText(frame, "X{0:.4f} Y{1:.4f} Z{2:.4f}".format(*Tpose.flatten()), (100,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0))
    
    cv2.imshow('img',frame)
    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break
        
cv2.destroyAllWindows() 

In [ ]:
# Release the capture and writer objects
cam.release()
cv2.destroyAllWindows() 